**Creación base de datos**


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
import scipy.integrate as integrate
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from astropy import constants as const
from astropy import units as u

## Defyning constanst 

In [ ]:
# Definiendo constantes
G = const.G.to(u.cm**3/(u.g*u.s**2)) # Constante Gravitacional en cm^3/(g*s^2)
Msol = const.M_sun.to(u.g) # Masa solar en gramos
R_sol = const.R_sun.to(u.cm)  # Radio del sol en centímetros

## Unidades

>- Temperature [K]
>- Mass [Earth Mass]
>- Radius [Earth Radius]
>- Distance [AU]
>- Metallicity [Fe/H]

**Open catalogue exoplanets**


>- Calcular flujo de radiación
> . Oblicuidad

In [ ]:
# Read the CSV file with space as the delimiter
exoplanet_data = pd.read_csv('exoplanet_data.csv', sep=',', comment='#')
exoplanet_data

,Planet_name,Host_name,gaia_id,sy_snum,GAIA_ID,Number_of_stars,number_of_Planets,Number of Moons,Discovery_reference,Discovery_facility,...,sy_disterr2,sy_vmag,sy_vmagerr1,sy_vmagerr2,sy_kmag,sy_kmagerr1,sy_kmagerr2,sy_gaiamag,sy_gaiamagerr1,sy_gaiamagerr2
0,K2-111 b,K2-111,Gaia DR2 53006669599267328,1,2,0,K2,0,5.351800,4.000000e-04,...,-2.69000,11.130,0.034,-0.034,9.377000,0.021,-0.021,10.92940,0.000621,-0.000621
1,TOI-1695 b,TOI-1695,Gaia DR2 534988616816537728,1,1,0,Transiting Exoplanet Survey Satellite (TESS),0,3.134279,7.100000e-06,...,-0.06805,12.989,0.081,-0.081,8.818000,0.021,-0.021,12.13640,0.000460,-0.000460
2,K2-170 b,K2-170,Gaia DR2 2599451994385832448,1,2,0,K2,0,7.576544,1.841000e-03,...,-6.29800,12.668,0.114,-0.114,11.075000,0.021,-0.021,12.49590,0.000222,-0.000222
3,Kepler-1038 c,Kepler-1038,Gaia DR2 2079028676836456960,1,2,0,Kepler,0,7.127660,NaN,...,-9.61800,15.030,0.229,-0.229,12.782000,0.022,-0.022,14.85610,0.000306,-0.000306
4,Kepler-1398 b,Kepler-1398,Gaia DR2 2128776801985730048,1,2,0,Kepler,0,2.788157,2.021000e-05,...,-10.60900,13.752,0.103,-0.103,12.355000,0.021,-0.021,13.66050,0.000228,-0.000228
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
700,Kepler-99 b,Kepler-99,Gaia DR2 2076871091425583232,2,1,0,Kepler,0,4.603580,NaN,...,-1.03100,13.206,0.092,-0.092,10.757000,0.018,-0.018,12.96180,0.000373,-0.000373
701,HIP 29442 d,HD 42813,Gaia DR2 2993561629444856960,1,3,0,Paranal Observatory,0,6.429575,2.600000e-05,...,-0.17630,9.490,0.030,-0.030,7.587000,0.024,-0.024,9.27468,0.000294,-0.000294
702,TOI-1467 b,TOI-1467,Gaia DR2 403141126645929728,1,1,0,Transiting Exoplanet Survey Satellite (TESS),0,5.971147,8.746000e-06,...,-0.05465,12.293,0.017,-0.017,8.568000,0.019,-0.019,11.56220,0.000875,-0.000875
703,EPIC 201595106 b,EPIC 201595106,Gaia DR2 3701123406596146048,1,2,0,K2,0,0.877240,4.000000e-05,...,-2.64600,11.746,0.069,-0.069,10.218000,0.021,-0.021,11.64770,0.000245,-0.000245


In [ ]:
header_1 = ['Planet_name', 'Host_name','GAIA_ID', 'Number_of_stars','number_of_Planets','Number of Moons','Discovery_reference', 'Discovery_facility','Orbital_Period_[days]', 'Orbit_Semi-Major_Axis_[au]','Planet_Radius_[Earth_Radius]','Planet_Radius_[Jupiter_Radius]','Planet_Mass_[Earth_mass]','Planet_Mass[Jupiter_mass]' ,'Planet_density[g/cm^3]','Eccentricity','Insolation_flux[Earth_flux]','Equilibrium_Temperature[K]','Inclination_[deg]','Ratio_of_Semi-Major Axis_to_Stellar_Radius','Ratio of Planet to Stellar Radius','Radial_Velocity_Amplitude_[m/s]','True_Obliquity','Spectral_Type','Stellar_Effective_Temperature_[K]','Stellar_Radius_[Solar Radius]','Stellar_Mass_[Solar mass]','Stellar_Metallicity_[dex]','Stellar_Metallicity_Ratio','Stellar_Luminosity_[log(Solar)]','Stellar_Surface_Gravity_[log10(cm/s**2)]','Stellar_Age_[Gyr]','Stellar_Rotational_Velocity_[km/s]','Stellar_Rotational_Period_[days]','RA_[deg]','Dec_[deg]','Distance_[pc]']

In [ ]:
header = exoplanet_data.columns.tolist()
selected_indices=[0,1,4,5,6,7,8,9,11,15,19,23,27,31,36,40,44,48,52,57,61,65,69,73,74,78,82,86,90,91,95,99,103,107,111,112,113]
selected_df = exoplanet_data.iloc[:, selected_indices]



In [ ]:
# Replace specific column names in the original DataFrame
for i, col_index in enumerate(selected_indices):
    exoplanet_data.columns.values[col_index] = header_1[i]

In [ ]:
exoplanet_data


,Planet_name,Host_name,gaia_id,sy_snum,GAIA_ID,Number_of_stars,number_of_Planets,Number of Moons,Discovery_reference,Discovery_facility,...,sy_disterr2,sy_vmag,sy_vmagerr1,sy_vmagerr2,sy_kmag,sy_kmagerr1,sy_kmagerr2,sy_gaiamag,sy_gaiamagerr1,sy_gaiamagerr2
0,K2-111 b,K2-111,Gaia DR2 53006669599267328,1,2,0,K2,0,5.351800,4.000000e-04,...,-2.69000,11.130,0.034,-0.034,9.377000,0.021,-0.021,10.92940,0.000621,-0.000621
1,TOI-1695 b,TOI-1695,Gaia DR2 534988616816537728,1,1,0,Transiting Exoplanet Survey Satellite (TESS),0,3.134279,7.100000e-06,...,-0.06805,12.989,0.081,-0.081,8.818000,0.021,-0.021,12.13640,0.000460,-0.000460
2,K2-170 b,K2-170,Gaia DR2 2599451994385832448,1,2,0,K2,0,7.576544,1.841000e-03,...,-6.29800,12.668,0.114,-0.114,11.075000,0.021,-0.021,12.49590,0.000222,-0.000222
3,Kepler-1038 c,Kepler-1038,Gaia DR2 2079028676836456960,1,2,0,Kepler,0,7.127660,NaN,...,-9.61800,15.030,0.229,-0.229,12.782000,0.022,-0.022,14.85610,0.000306,-0.000306
4,Kepler-1398 b,Kepler-1398,Gaia DR2 2128776801985730048,1,2,0,Kepler,0,2.788157,2.021000e-05,...,-10.60900,13.752,0.103,-0.103,12.355000,0.021,-0.021,13.66050,0.000228,-0.000228
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
700,Kepler-99 b,Kepler-99,Gaia DR2 2076871091425583232,2,1,0,Kepler,0,4.603580,NaN,...,-1.03100,13.206,0.092,-0.092,10.757000,0.018,-0.018,12.96180,0.000373,-0.000373
701,HIP 29442 d,HD 42813,Gaia DR2 2993561629444856960,1,3,0,Paranal Observatory,0,6.429575,2.600000e-05,...,-0.17630,9.490,0.030,-0.030,7.587000,0.024,-0.024,9.27468,0.000294,-0.000294
702,TOI-1467 b,TOI-1467,Gaia DR2 403141126645929728,1,1,0,Transiting Exoplanet Survey Satellite (TESS),0,5.971147,8.746000e-06,...,-0.05465,12.293,0.017,-0.017,8.568000,0.019,-0.019,11.56220,0.000875,-0.000875
703,EPIC 201595106 b,EPIC 201595106,Gaia DR2 3701123406596146048,1,2,0,K2,0,0.877240,4.000000e-05,...,-2.64600,11.746,0.069,-0.069,10.218000,0.021,-0.021,11.64770,0.000245,-0.000245


In [ ]:
# Strip whitespace and fix hidden characters
exoplanet_data.columns = exoplanet_data.columns.str.strip()
exoplanet_data.columns = exoplanet_data.columns.str.replace('\u200b', '', regex=False)
exoplanet_data['Planet_name']

0              K2-111 b
1            TOI-1695 b
2              K2-170 b
3         Kepler-1038 c
4         Kepler-1398 b
             ...       
700         Kepler-99 b
701         HIP 29442 d
702          TOI-1467 b
703    EPIC 201595106 b
704         TOI-1798.02
Name: Planet_name, Length: 705, dtype: object

In [ ]:
cambios_realizados = 0
guardar_cada = 5
historial = []

for i in header_1:
    if exoplanet_data[i].isna().any():
        print(f"🔍 Revisando columna: {i}")

        for idx in exoplanet_data[exoplanet_data[i].isna()].index:
            planeta = exoplanet_data.at[idx, 'Planet_name'] if 'Planet_name' in exoplanet_data.columns else f"Fila {idx}"
            print(f"🌍 Planeta: {planeta} | Columna: {i}")

            while True:
                r = input("¿Es diferente a NaN? (Y/N): ").strip().upper()
                if r in ['Y', 'N']:
                    break
                print("⚠️ Entrada no válida. Por favor escribe 'Y' o 'N'.")

            if r == 'Y':
                valor = input(f"Ingrese un valor para '{i}' en fila {idx} (o escribe 'volver' para deshacer el último cambio): ").strip()

                if valor.lower() == "volver":
                    if historial:
                        fila_ant, col_ant, val_ant = historial.pop()
                        exoplanet_data.at[fila_ant, col_ant] = val_ant
                        cambios_realizados -= 1
                        print(f"↩️ Último cambio deshecho en fila {fila_ant}, columna '{col_ant}'.")
                        continue
                    else:
                        print("⚠️ No hay cambios anteriores para deshacer.")
                        continue

                valor_anterior = exoplanet_data.at[idx, i]
                historial.append((idx, i, valor_anterior))

                exoplanet_data.at[idx, i] = valor
                cambios_realizados += 1
                print(f"✅ Valor guardado: {valor}")

            elif r == 'N':
                exoplanet_data.at[idx, i] = np.nan
                cambios_realizados += 1
                print("🔁 Se mantiene como NaN.")

            # Guardado automático
            if cambios_realizados % guardar_cada == 0:
                exoplanet_data.to_csv("exoplanet_data.csv", index=False)
                print(f" {cambios_realizados} cambios guardados en el archivo.")

# Guardado final


In [ ]:
cambios_realizados = 0
guardar_cada = 5
reiniciar = False
historial = []  # Para registrar los cambios y permitir deshacer

for i in header_1:
    if exoplanet_data[i].isna().any():
        for j in exoplanet_data[i]:
            r = input(f"¿Es diferente NaN? (Y/N): ").strip().upper()
            if r == 'Y':
                valor = input(f"Ingrese un valor para '{i}': ").strip()
                exoplanet_data[i].fillna(valor, inplace=True)
                cambios_realizados += 1
                print(f"✅ Valor guardado: {valor}")
            elif r == 'N':
                exoplanet_data[i].fillna(np.nan, inplace=True)
                cambios_realizados += 1
                print("🔁 Se mantiene como NaN.")
            else:
                print("⚠️ Entrada no válida. Por favor escribe 'Y' o 'N'.")
    


▶️ Continuando...
▶️ Continuando...
▶️ Continuando...
